In [1]:
pip install nexcsi


  Using cached nexcsi-0.5.2-py3-none-any.whl.metadata (2.4 kB)
Using cached nexcsi-0.5.2-py3-none-any.whl (8.6 kB)
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------  15.5/15.5 MB 74.9 MB/s eta 0:00:01
   ---------------------------------------- 15.5/15.5 MB 61.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from nexcsi import decoder

device = "raspberrypi" # nexus5, nexus6p, rtac86u

samples = decoder(device).read_pcap('C:/Users/woons/OneDrive/바탕 화면/data/losdown1.pcap')

print(samples['rssi']) # [-75 -77 -77 ... -77 -76 -76]
print(samples['fctl']) # [128 148 148 ... 148 148 148]
print(samples['csi'])  # [[ 19489  0  -19200  -96 -42 ...

# samples is a Numpy Structured Array
print(samples.dtype)

# [
#     ('ts_sec', '<u4'), ('ts_usec', '<u4'), ('saddr', '>u4'), 
#     ('daddr', '>u4'), ('sport', '>u2'), ('dport', '>u2'),
#     ('magic', '<u2'), ('rssi', 'i1'), ('fctl', 'u1'),
#     ('mac', 'u1', (6,)), ('seq', '<u2'), ('css', '<u2'),
#     ('csp', '<u2'), ('cvr', '<u2'), ('csi', '<i2', (512,))
# ]

# Accessing CSI as type complex64
csi = decoder(device).unpack(samples['csi'])

[-42 -42 -45 -42 -47 -45 -45 -45 -41 -45 -44 -41 -44 -40 -44 -44 -47 -40
 -41 -41]
[136 136 136 136 136 136 136 136 136 136 136 136 136 136 136 136 136 136
 136 136]
[[-321 -192  728 ...  812  495  781]
 [-321 -192 -434 ...  451 -715  590]
 [-321 -192  542 ...  789   92  859]
 ...
 [-321 -192 -480 ...  604 -814  784]
 [-321 -192  507 ... -547  752 -697]
 [-321 -192  969 ...  675  796  564]]
[('ts_sec', '<u4'), ('ts_usec', '<u4'), ('saddr', '>u4'), ('daddr', '>u4'), ('sport', '>u2'), ('dport', '>u2'), ('magic', '<u2'), ('rssi', 'i1'), ('fctl', 'u1'), ('mac', 'u1', (6,)), ('seq', '<u2'), ('css', '<u2'), ('csp', '<u2'), ('cvr', '<u2'), ('csi', '<i2', (128,))]


In [14]:
from nexcsi import decoder
import numpy as np
device = "raspberrypi" # nexus5, nexus6p, rtac86u

samples = decoder(device).read_pcap('C:/Users/woons/OneDrive/바탕 화면/data/losdown1.pcap')

csi = decoder(device).unpack(samples['csi'], zero_nulls=True, zero_pilots=True)
#csi = np.delete(csi, csi.dtype.metadata['nulls'], axis=1)
#csi = np.delete(csi, csi.dtype.metadata['pilots'], axis=1)
# samples is a Numpy Structured Array
print(csi[1])



[    0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j  -537. -246.j
  -560.  +98.j  -385. +415.j  -130. +540.j    94. +554.j   304. +483.j
   440. +363.j     0.   +0.j   564.  +66.j   539. -100.j   509. -221.j
   471. -389.j   368. -521.j   236. -606.j   116. -680.j   -28. -676.j
  -148. -687.j  -308. -664.j  -440. -602.j  -573. -513.j  -690. -417.j
     0.   +0.j  -813. -138.j  -839.  +15.j  -829. +146.j  -820. +301.j
  -776. +451.j  -715. +590.j     0.   +0.j  -434. +789.j  -296. +811.j
  -131. +827.j    12. +811.j   157. +767.j   287. +700.j     0.   +0.j
   483. +505.j   545. +381.j   575. +267.j   587. +147.j   574.  +17.j
   535.  -97.j   465. -200.j   377. -277.j   289. -331.j   190. -362.j
    88. -374.j   -11. -367.j  -109. -328.j     0.   +0.j  -249. -194.j
  -287.  -95.j  -295.  +20.j  -250. +138.j  -159. +228.j   -15. +260.j
 10246.+5376.j     0.   +0.j     0.   +0.j     0.   +0.j]


In [23]:
from nexcsi import decoder
import numpy as np

device = "raspberrypi"  # nexus5, nexus6p, rtac86u

# PCAP 파일 읽기
samples = decoder(device).read_pcap('C:/Users/woons/OneDrive/바탕 화면/data/losup1.pcap')

# CSI 데이터를 언팩하기
csi = decoder(device).unpack(samples['csi'], zero_nulls=True, zero_pilots=True)

# dB 진폭을 저장할 리스트 초기화
db_amplitudes_list = []

# CSI 데이터에서 각 csi[0], csi[1], ..., csi[19]에 대해 반복
for index in range(len(csi)):  # csi[0]부터 csi[19]까지 반복
    csi_filtered = csi[index][(csi[index].real != 0) | (csi[index].imag != 0)]  # 0.0 + 0.j를 제외한 나머지 값 선택

    if csi_filtered.size > 0:  # 유효한 데이터가 있는 경우에만 처리
        amplitudes = np.abs(csi_filtered)  # 진폭 계산
        db_values = 20 * np.log10(np.where(amplitudes > 0, amplitudes, np.nan))  # dB로 변환
        db_amplitudes_list.append(db_values)  # dB 값을 리스트에 추가

# 결과를 CSV 파일로 저장
output_file_path = 'C:/Users/woons/OneDrive/바탕 화면/data/test1.csv'
with open(output_file_path, 'w') as f:
    for db_amplitudes in db_amplitudes_list:
        f.write(','.join(map(str, db_amplitudes)) + '\n')  # 각 행에 dB 값을 저장
